In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS     = 40
BATCH_SIZE = 64

In [37]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)




60000

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x,2)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.conv2_drop(x)
        x = F.relu(F.max_pool2d(x, 2))
        
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [13]:
dataiter = iter(train_loader)
images, labels = next(dataiter)

images.size()

torch.Size([64, 1, 28, 28])

In [30]:
                    # input_data = torch.randn(64, 1, 28, 28)  # 무작위 입력 이미지 생성
                    # conv_layer = nn.Conv2d(1, 10, kernel_size=5)
                    # conv_layer2 = nn.Conv2d(10, 20, kernel_size=5)
                    # conv2_drop = nn.Dropout2d()



                    # output = conv_layer(input_data)
                    # output2 = conv_layer2(output)
                    # print(input_data.size())
                    # print(output.size())
                    # print(output2.size())

torch.Size([64, 1, 28, 28])
torch.Size([64, 10, 24, 24])
torch.Size([64, 20, 20, 20])


In [31]:
dropout_layer = nn.Dropout2d(p=0.5)

# 입력 데이터 생성 (예: 1 배치, 1 채널, 28x28 이미지)
input_data = torch.randn(1, 1, 28, 28)

# 드롭아웃을 적용하여 출력을 얻습니다.
output = dropout_layer(input_data)

print("입력 데이터 크기:", input_data.size())
print("출력 데이터 크기:", output.size())

입력 데이터 크기: torch.Size([1, 1, 28, 28])
출력 데이터 크기: torch.Size([1, 1, 28, 28])


In [32]:

model     = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [33]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [34]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [35]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.321073
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.099284
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.723199
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.381006
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.474939
[1] Test Loss: 0.1727, Accuracy: 94.88%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.296927
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.447677
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.228267
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.509794
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.393279
[2] Test Loss: 0.1171, Accuracy: 96.45%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.284285
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.315386
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.240039
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.251048
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.328180
[3] Test Loss: 0.0947, Accuracy: 97.05%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.268389
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.116788
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.202535


In [ ]:
class abc:
    qwewqe: "wqewq"

    def __init__(self, a , b):
        self.a 